In [1]:
# =============================================================================
# CONSUMPTION 18: MALAWI VILLAGE
# =============================================================================

import numpy as np
import pandas as pd
import os 
os.chdir('C:/Users/rodri/Dropbox/PhD thesis/python')
from data_functions_albert import remove_outliers, gini
os.chdir('C:/Users/rodri/Dropbox/Chiefs_Field_Aug_18/Data/')
percentiles = [0.05, 0.1, .25, .5, .75, 0.8, 0.9, 0.95, 0.99]
euro_MWK = 806.64


#Import data
data = pd.read_csv("data_SIEG.csv")

x0 = data.columns.get_loc("consume_1")
x1 = data.columns.get_loc("salt_unit9")
datacon = data.iloc[:,x0:x1]

list_items = ['maizemgaiwa', 'maizerefined', 'maizemadeya', 'maizegrain', 'greenmaize', 'rice', 'cassavatubers', 'wsweetpotatoes', 'osweetpotatoes'
, 'ipotatoes', 'potatocrisps', 'bbean', 'pigeonpea', 'groundnut', 'groundnutf', 'onion', 'cabbage', 'tanaposi', 'leafyvegetables', 'tomato', 'eggs', 
'driedfish', 'fleshfish', 'goat', 'chicken', 'otherpoultry', 'smockedfish', 'mango', 'banana', 'guava', 
'wildfruits',  'sugar', 'sugarcane', 'cookingoil', 'softdrinks',
'thobwa', 'locallybrewed', 'salt']


#Obtain the names of the variables per each question of item. 
#Question c is monetary question so not conversion to kgs needed
a_var = []
b_var = []
d_var = []
e_var = []
f_var = []
g_var = []
h_var = []
i_var = []
j_var = []

#Generate variable lables in a list
for item in list_items :
    a = item+'_a'
    b= item+'_b'
    d = item+'_d'
    e = item+'_e'
    f = item+'_f'
    g = item+'_g'
    h = item+'_h'
    i = item+'_i'
    j = item+'_j'
    a_var.append(a)
    b_var.append(b)
    d_var.append(d)
    e_var.append(e)
    f_var.append(f)
    g_var.append(g)
    h_var.append(h)
    i_var.append(i)
    j_var.append(j) 


list_questions = ['a','b','d','e','f','g','h','i','j']

# Generate kg variables empty
for item in list_items:
    for q in list_questions:
        datacon[item+'_'+q+'kg']= np.nan
    
# IN UNITS QUESTIONS: Drop nan observations. Also drop unit 25 (number not in our choices). Also drop 24(whole chicken).
for var in list_items:
    for i in range(1,9):
        datacon[[var+'_unit'+str(i)]] = datacon[[var+'_unit'+str(i)]].replace(np.nan, 99)
        datacon[[var+'_unit'+str(i)]] = datacon[[var+'_unit'+str(i)]].replace(25, 99)
        datacon[[var+'_unit'+str(i)]] = datacon[[var+'_unit'+str(i)]].replace(24, 99)

## For salt we don't have question of units for given-out ganyu:
for var in list_items[0:-1]:
    datacon[[var+'_unit9']] = datacon[[var+'_unit9']].replace(np.nan, 99)
    datacon[[var+'_unit9']] = datacon[[var+'_unit9']].replace(25, 99)
    datacon[[var+'_unit9']] = datacon[[var+'_unit9']].replace(24, 99)        

'''
##### THIS CODE IS TO GENERATE THE MATRIX OF ITEMS/UNITS COMBINATIONS TO CONVERT TO KGS. NO NEED TO RE-RUN IT.
data_conversion = pd.read_excel('conversionkgrates_consumption_malawi_short.xls')

#del data, datasets, data0, data1, data2, data3, data4, x0, x1

items_conv = data_conversion.item_label.unique()


list_items_conv = ['Maize Mgaiwa', 'Maize refined', 'Maize Madeya', 'Maize grain',
       'Greenmaize', 'Rice', 'Cassava tubers',
       'White sweet potatoes', 'Orange sweet potatoes', 'Irish Potatoe',
       'Potatoe Chips', 'Bean, brown', 'Pigeon peas (ndolo)', 'Groundnut',
       'Groundnut flour', 'Onion', 'Cabbage', 'Tanaposi/Rape',
       'Other cultivated green leafy vegetables', 'Tomatoe', 'Eggs',
       'Dried Fish', 'Fresh Fish', 'Goat', 'Chicken',
       'Other poultry-guinea fowl, doves, etc', 'Smoked Fish', 'Mango',
       'Banana', 'Guava', 'Wild fruit (Masau, Malambe, etc)', 'Sugar',
       'Sugar Cane', 'Cooking oil',
       'Soft drinks(coca-cola, fanta, sprite etc)', 'Thobwa',
       'Locally brewed liquor(kachasu)', 'Salt']


#Obtain the names of the variables per each question of item. Question c is monetary question so not conversion to kgs needed
a_var = []
b_var = []
d_var = []
e_var = []
f_var = []
g_var = []
h_var = []
i_var = []
j_var = []
#Generate variable lables in a list
for item in list_items :
    a = item+'_a'
    b= item+'_b'
    d = item+'_d'
    e = item+'_e'
    f = item+'_f'
    g = item+'_g'
    h = item+'_h'
    i = item+'_i'
    j = item+'_j'
    a_var.append(a)
    b_var.append(b)
    d_var.append(d)
    e_var.append(e)
    f_var.append(f)
    g_var.append(g)
    h_var.append(h)
    i_var.append(i)
    j_var.append(j) 


list_questions = ['a','b','d','e','f','g','h','i','j']

# Generate kg variables empty
for item in list_items:
    for q in list_questions:
        datacon[item+'_'+q+'kg']= np.nan
    
# Drop nan observations. Also drop unit 25 (number not in our choices). Also drop 24(whole chicken).
for var in list_items:
    for i in range(1,9):
        datacon[[var+'_unit'+str(i)]] = datacon[[var+'_unit'+str(i)]].replace(np.nan, 99)
        datacon[[var+'_unit'+str(i)]] = datacon[[var+'_unit'+str(i)]].replace(25, 99)
        datacon[[var+'_unit'+str(i)]] = datacon[[var+'_unit'+str(i)]].replace(24, 99)
        
#       
# Import conversion files
conversionkg = data_conversion[['unit','item_label','conversion_kgs_country']]


# Reshape as: rows:units, columns:crops

conversionkg = conversionkg.replace(list_items_conv, list_items)

conversionkg_pivot = conversionkg.pivot_table(values='conversion_kgs_country',
                                index='unit',
                                columns='item_label')


conversionkg_pivot.loc[99,:] = np.nan

conversionkg_pivot.to_csv('conversionkg_isaprices_matrix.csv')
'''

### CONVERSION TO KGS: ISA-LSMS SOUTH MALAWI PRICE CONVERSION WITH ALBERT REPLACING VALUES THAT DIDN'T HAVE COMMON SENSE
conversionkg_pivot = pd.read_csv('conversionkg_isaprices_matrix_2.csv',  index_col=0)


C:\Users\rodri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (62,78,86,122,124,133,160,191,197,222,253,259,315,377,408,412,424,433,439,443,459,569,600,625,657,796,797,810,815,827,830,832,850,852,869,871,892,902,906,929,933,947,951,954,955,958,962,965,969,978,982,985,986,989,992,996,1003,1006,1023,1040,1187,1219,1285,1286,1295,1296,1305,1306,1324,1328,1345,1349,1386,1389,1390,1425,1435,1444,1460,1462,1491,1498,1508,1517,1526,1533,1535,1588,2384,2387,2524,2527,2531,2823,2826,3012,3021,3022,3027,3034,3083,3092,3093,3105,3154,3164,3228,3234,3235,3237,3240,3242,3249,3258,3259,3264,3319,3328,3386,3389,3390,3396,3398,3399,3462,3465,3468,3470,3502,3508,3510,3532,3533,3534,3535,3561,3562,3572,3575,3576,3577,3591,3594,3595,3610,3613,3614,3629,3632,3648,3652,3663,3673,3674,3683,3684,3694,3695,3710,3715,3720,3725,3730,3735,3740,3741,3751,3752,3762,3766,3767,3780,3784,3785,3798,3802,3808) have mixed types. Specify dtype option on import or set low_memory=F

In [2]:

print('a: Total Consumption')
for var in a_var:
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit1')]), item]
    print(datacon[[var+'kg']].describe())


a: Total Consumption
       maizemgaiwa_akg
count       197.000000
mean          1.461422
std           3.094568
min           0.001000
25%           0.300000
50%           0.480000
75%           1.200000
max          20.000000
       maizerefined_akg
count        121.000000
mean           1.840143
std            4.731409
min            0.001000
25%            0.300000
50%            0.300000
75%            1.000000
max           30.000000
       maizemadeya_akg
count        75.000000
mean          0.419898
std           0.436218
min           0.001000
25%           0.300000
50%           0.300000
75%           0.416667
max           3.000000
       maizegrain_akg
count       57.000000
mean         0.443977
std          0.675216
min          0.120000
25%          0.166667
50%          0.300000
75%          0.300000
max          5.000000
       greenmaize_akg
count       12.000000
mean         1.383750
std          2.742309
min          0.000000
25%          0.300000
50%          0.6000

In [3]:
print('b: Bought')
for var in b_var:
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit2')]), item]

print('d: Own-produced')
for var in d_var:
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit3')]), item]

print('e: Gift-in')
for var in e_var:
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit4')]), item]

print('f: Gift-in for free')
for var in f_var:
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit5')]), item]

print('g: Gift-in for ganyu')
for var in g_var:
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit6')]), item]

print('h: Gift-out')
for var in h_var:
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit7')]), item]

print('i: Food giften out for free')
for var in i_var:
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit8')]), item]

print('j: Food giften out for ganyu')
for var in j_var:
    for i in range(len(datacon)): 
        datacon.iloc[i,datacon.columns.get_loc(var+'kg')] = datacon.iloc[i,datacon.columns.get_loc(var)]*conversionkg_pivot.loc[int(datacon.iloc[i,  datacon.columns.get_loc(item+'_unit9')]), item]



b: Bought
d: Own-produced
e: Gift-in
f: Gift-in for free
g: Gift-in for ganyu
h: Gift-out
i: Food giften out for free
j: Food giften out for ganyu


KeyError: 'salt_unit9'

In [4]:
print(datacon[['bbean_a', 'bbean_unit1', 'bbean_akg','bbean_b', 'bbean_bkg', 'bbean_d', 'bbean_dkg' ]])
 


     bbean_a  bbean_unit1  bbean_akg  bbean_b  bbean_bkg  bbean_d  bbean_dkg
0        NaN         99.0        NaN      NaN        NaN      NaN        NaN
1        NaN         99.0        NaN      NaN        NaN      NaN        NaN
2        3.0         16.0       0.72      0.0        0.0      0.0        NaN
3        NaN         99.0        NaN      NaN        NaN      NaN        NaN
4        1.0          7.0       0.24      0.0        0.0      0.0        NaN
..       ...          ...        ...      ...        ...      ...        ...
264      NaN         99.0        NaN      NaN        NaN      NaN        NaN
265      NaN         99.0        NaN      NaN        NaN      NaN        NaN
266      NaN         99.0        NaN      NaN        NaN      NaN        NaN
267      NaN         99.0        NaN      NaN        NaN      NaN        NaN
268      NaN         99.0        NaN      NaN        NaN      NaN        NaN

[269 rows x 7 columns]


In [5]:
#%% Checks

# Sum the subset questions
for item in list_items:
    datacon[item+'_akg_check'] = datacon[item+'_bkg'] + datacon[item+'_dkg'] + datacon[item+'_ekg']
    datacon[item+'_ekg_check'] = datacon[item+'_fkg'] + datacon[item+'_gkg']
    datacon[item+'_hkg_check'] = datacon[item+'_ikg'] + datacon[item+'_jkg']


# Check the correlation of totals wrt sum of subset.
corr_total = []
corr_gift = []
corr_given = []
for item in list_items: 
    a = datacon[item+'_akg'].corr(datacon[item+'_akg_check'])
    b = datacon[item+'_ekg'].corr(datacon[item+'_ekg_check'])
    c = datacon[item+'_hkg'].corr(datacon[item+'_hkg_check'])
    corr_total.append(a)
    corr_gift.append(b)
    corr_given.append(c)
    #print(datacon[[item+'_akg',item+'_akg_check']])
    #print(datacon[[item+'_ekg',item+'_ekg_check']])
    #print(datacon[[item+'_hkg',item+'_hkg_check']])
    
    

C:\Users\rodri\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3175: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\rodri\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3109: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


In [6]:
#%% Checks

# Sum the subset questions


#check total consumption
datacon['total_foodkg'] = 0
datacon['purchased_kg'] = 0
datacon['ownproduced_kg'] = 0
datacon['giftin_kg'] = 0
datacon['giftin_free_kg'] = 0
datacon['giftin_ganyu_kg'] = 0
datacon['giftout_kg'] = 0
datacon['giftout_free_kg'] = 0
datacon['giftout_ganyu_kg'] = 0


for item in list_items:
    datacon['total_foodkg'] += datacon[item+'_akg'].replace(np.nan, 0)
    datacon['purchased_kg'] += datacon[item+'_bkg'].replace(np.nan, 0)
    datacon['ownproduced_kg'] += datacon[item+'_dkg'].replace(np.nan, 0)
    datacon['giftin_kg'] += datacon[item+'_ekg'].replace(np.nan, 0)
    datacon['giftin_free_kg'] += datacon[item+'_fkg'].replace(np.nan, 0)
    datacon['giftin_ganyu_kg'] += datacon[item+'_gkg'].replace(np.nan, 0)
    datacon['giftout_kg'] += datacon[item+'_hkg'].replace(np.nan, 0)
    datacon['giftout_free_kg'] += datacon[item+'_ikg'].replace(np.nan, 0)
    datacon['giftout_ganyu_kg'] += datacon[item+'_jkg'].replace(np.nan, 0)
    
sumtotalfoodkg = datacon[['total_foodkg', 'purchased_kg','ownproduced_kg', 'giftin_kg','giftin_free_kg','giftin_ganyu_kg','giftout_kg','giftout_free_kg','giftout_ganyu_kg']].describe(percentiles)   


print('==== Summary Food Consumption last 7 days in kgs aggregated across items =======')
print('')
print(sumtotalfoodkg)

==== Summary Food Consumption last 7 days in kgs aggregated across items =======

       total_foodkg  purchased_kg  ownproduced_kg   giftin_kg  giftin_free_kg  \
count    269.000000    269.000000           269.0  269.000000      269.000000   
mean      54.622282     39.366084             0.0    0.395511        1.158039   
std      231.053623    195.488795             0.0    5.715449       18.717441   
min        0.000000      0.000000             0.0    0.000000        0.000000   
5%         0.000000      0.000000             0.0    0.000000        0.000000   
10%        0.532600      0.257400             0.0    0.000000        0.000000   
25%        6.000000      2.700000             0.0    0.000000        0.000000   
50%       11.550000      6.666667             0.0    0.000000        0.000000   
75%       34.350000     18.500000             0.0    0.000000        0.000000   
80%       53.040001     30.600000             0.0    0.000000        0.000000   
90%      103.620000     80.

In [7]:

# Get prices -------------------------
for item in list_items:
        datacon[item+'_price']= np.nan

## Compute price paid per each household each item
for item in list_items:
    datacon[item+'_price'] = datacon[item+'_c'] / datacon[item+'_bkg'].replace(0,np.nan)

## Get median prices per item
for item in list_items:
    print('Median Price 1 kg of '+item)
    datacon['med_price_'+item] = datacon[item+'_price'].median()
    print(datacon['med_price_'+item].mean())


Median Price 1 kg of maizemgaiwa
2666.666666666662
Median Price 1 kg of maizerefined
2666.666666666662
Median Price 1 kg of maizemadeya
750.0
Median Price 1 kg of maizegrain
425.0
Median Price 1 kg of greenmaize
250.00000000000006
Median Price 1 kg of rice
725.0
Median Price 1 kg of cassavatubers
166.66666666666637
Median Price 1 kg of wsweetpotatoes
400.0
Median Price 1 kg of osweetpotatoes
599.9999807999995
Median Price 1 kg of ipotatoes
666.6666666666655
Median Price 1 kg of potatocrisps
1000.0
Median Price 1 kg of bbean
1000.0000000000002
Median Price 1 kg of pigeonpea
416.6666666666681
Median Price 1 kg of groundnut
333.33333333333275
Median Price 1 kg of groundnutf
333.33333333333275
Median Price 1 kg of onion
166.66666666666637
Median Price 1 kg of cabbage
500.0
Median Price 1 kg of tanaposi
166.66666666666637
Median Price 1 kg of leafyvegetables
166.66666666666637
Median Price 1 kg of tomato
166.66666666666637
Median Price 1 kg of eggs
400.0
Median Price 1 kg of driedfish
333.3

In [8]:

## compute consumption in MWK -----------------   
for item in list_items:
    for q in list_questions:
        datacon[item+'_'+q+'MWK']= np.nan

print('a: Total Consumption')
for item in list_items:
    datacon[item+'_aMWK'] = datacon[item+'_akg']*datacon['med_price_'+item]
    print('Food Consumption in MWK during last 7 days item: '+item)
    print(datacon[item+'_aMWK'].describe(percentiles=percentiles))
            
print('b: Bought')
for item in list_items:
    datacon[item+'_bMWK'] = datacon[item+'_bkg']*datacon['med_price_'+item]


print('d: Own-produced')
for item in list_items:
    datacon[item+'_dMWK'] = datacon[item+'_dkg']*datacon['med_price_'+item]
    

print('e: Gift-in')
for item in list_items:
    datacon[item+'_dMWK'] = datacon[item+'_dkg']*datacon['med_price_'+item]
    
print('f: Gift-in for free')
for item in list_items:
    datacon[item+'_fMWK'] = datacon[item+'_fkg']*datacon['med_price_'+item]

print('g: Gift-in for ganyu')
for item in list_items:
    datacon[item+'_gMWK'] = datacon[item+'_gkg']*datacon['med_price_'+item]
    

print('h: Gift-out')
for item in list_items:
    datacon[item+'_hMWK'] = datacon[item+'_hkg']*datacon['med_price_'+item]
    
    
print('i: Food giften out for free')
for item in list_items:
    datacon[item+'_iMWK'] = datacon[item+'_ikg']*datacon['med_price_'+item]
 
    
print('j: Food giften out for ganyu')
for item in list_items:
    datacon[item+'_jMWK'] = datacon[item+'_jkg']*datacon['med_price_'+item]    
    

a: Total Consumption
Food Consumption in MWK during last 7 days item: maizemgaiwa
count      197.000000
mean      3897.125767
std       8252.182065
min          2.666667
5%           5.333333
10%        400.000000
25%        800.000000
50%       1279.999973
75%       3199.999933
80%       4000.000000
90%       8000.000000
95%      13866.666667
99%      53333.333333
max      53333.333333
Name: maizemgaiwa_aMWK, dtype: float64
Food Consumption in MWK during last 7 days item: maizerefined
count      121.000000
mean      4907.048663
std      12617.091147
min          2.666667
5%           2.666667
10%         66.666667
25%        800.000000
50%        800.000000
75%       2666.666667
80%       2666.666667
90%       8000.000000
95%      26666.666667
99%      64000.000000
max      80000.000000
Name: maizerefined_aMWK, dtype: float64
Food Consumption in MWK during last 7 days item: maizemadeya
count      75.000000
mean      314.923333
std       327.163215
min         0.750000
5%          0.75

In [9]:
    
#check total consumption
datacon['c_food'] = 0
datacon['c_food_purch'] = 0
datacon['c_food_ownprod'] = 0
datacon['c_food_giftin'] = 0
datacon['c_food_giftin_free'] = 0
datacon['c_food_giftin_ganyu'] = 0
datacon['c_food_giftout'] = 0
datacon['c_food_giftout_free'] = 0
datacon['c_food_giftout_ganyu'] = 0

for item in list_items:
    datacon['c_food'] += datacon[item+'_aMWK'].replace(np.nan, 0)
    datacon['c_food_purch'] += datacon[item+'_bMWK'].replace(np.nan, 0)
    datacon['c_food_ownprod'] += datacon[item+'_dMWK'].replace(np.nan, 0)
    datacon['c_food_giftin'] += datacon[item+'_eMWK'].replace(np.nan, 0)
    datacon['c_food_giftin_free'] += datacon[item+'_fMWK'].replace(np.nan, 0)
    datacon['c_food_giftin_ganyu'] += datacon[item+'_gMWK'].replace(np.nan, 0)
    datacon['c_food_giftout'] += datacon[item+'_hMWK'].replace(np.nan, 0)
    datacon['c_food_giftout_free'] += datacon[item+'_iMWK'].replace(np.nan, 0)
    datacon['c_food_giftout_ganyu'] += datacon[item+'_jMWK'].replace(np.nan, 0)

datacon[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu']] = datacon[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu']].replace(0.0,np.nan)

 
sumcfood= ((datacon[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu']]*4/euro_MWK).replace(0.0,np.nan)).describe(percentiles)  
 
print('==== Summary Food Consumption at Month level in Euros =======')
print(sumcfood)


==== Summary Food Consumption at Month level in Euros =======
             c_food  c_food_purch  c_food_ownprod  c_food_giftin  \
count    254.000000    249.000000             0.0            0.0   
mean     192.895067    157.520802             NaN            NaN   
std     1569.808521   1541.650195             NaN            NaN   
min        0.218602      0.045318             NaN            NaN   
5%         0.804560      0.632087             NaN            NaN   
10%        4.145591      1.287646             NaN            NaN   
25%       15.962511      8.306060             NaN            NaN   
50%       33.701527     17.236933             NaN            NaN   
75%       73.838322     49.078758             NaN            NaN   
80%      117.128665     65.896255             NaN            NaN   
90%      269.500645    155.995515             NaN            NaN   
95%      355.275174    280.869616             NaN            NaN   
99%     1060.431622    891.789191             NaN     

In [10]:
#%%  Export dataset

## Convert to rainy season (7 months)
datacon[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu','total_foodkg', 'purchased_kg','ownproduced_kg', 'giftin_kg','giftin_free_kg','giftin_ganyu_kg','giftout_kg','giftout_free_kg','giftout_ganyu_kg']] = datacon[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu','total_foodkg', 'purchased_kg','ownproduced_kg', 'giftin_kg','giftin_free_kg','giftin_ganyu_kg','giftout_kg','giftout_free_kg','giftout_ganyu_kg']]*(4*7)

datacon_short = datacon[[ 'hhid','c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu','total_foodkg', 'purchased_kg','ownproduced_kg', 'giftin_kg','giftin_free_kg','giftin_ganyu_kg','giftout_kg','giftout_free_kg','giftout_ganyu_kg']] = datacon[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu','total_foodkg', 'purchased_kg','ownproduced_kg', 'giftin_kg','giftin_free_kg','giftin_ganyu_kg','giftout_kg','giftout_free_kg','giftout_ganyu_kg']]

## If file used is "isaprices_matrix" then cvgtokg: conversion to kg south malawi using prices.
## If file used is "isaprices_matrix_2" then cvgtokg2: Albert modification of conversion rate that didn't have sense.
datacon_short.to_csv('cons_short_18_cvgtokg2.csv', index=False)



ValueError: Columns must be same length as key

In [11]:
#%%  Export dataset

## Convert to rainy season (7 months)
datacon[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu','total_foodkg', 'purchased_kg','ownproduced_kg', 'giftin_kg','giftin_free_kg','giftin_ganyu_kg','giftout_kg','giftout_free_kg','giftout_ganyu_kg']] = datacon[['c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu','total_foodkg', 'purchased_kg','ownproduced_kg', 'giftin_kg','giftin_free_kg','giftin_ganyu_kg','giftout_kg','giftout_free_kg','giftout_ganyu_kg']]*(4*7)

datacon_short = datacon[[ 'hhid','c_food', 'c_food_purch' , 'c_food_ownprod', 'c_food_giftin', 'c_food_giftin_free', 'c_food_giftin_ganyu', 'c_food_giftout', 'c_food_giftout_free', 'c_food_giftout_ganyu','total_foodkg', 'purchased_kg','ownproduced_kg', 'giftin_kg','giftin_free_kg','giftin_ganyu_kg','giftout_kg','giftout_free_kg','giftout_ganyu_kg']] 
## If file used is "isaprices_matrix" then cvgtokg: conversion to kg south malawi using prices.
## If file used is "isaprices_matrix_2" then cvgtokg2: Albert modification of conversion rate that didn't have sense.
datacon_short.to_csv('cons_short_18_cvgtokg2.csv', index=False)



KeyError: "['hhid'] not in index"